In [36]:
from ortools.linear_solver import pywraplp
from gen import gen_average_degree_directed, gen_planted_path, StandardGraph
import time

In [37]:
solver = pywraplp.Solver.CreateSolver("CP-SAT")

n = 100
graph = gen_average_degree_directed(n, n)

N = graph.vertices
M = N

adjacency_matrix = graph.to_matrix()
variables = [[solver.IntVar(0.0, 1.0, f"X[{place}, {vert}]") for vert in range(N + 1)] for place in range(M)]

# Define cost function
cost = sum([sum([vars[i] for i in range(N)]) for vars in variables])

solver.Maximize(cost)

# No repeated vertices restriction
for vert in range(N):

    exp = 0

    for place in range(M):
        exp += variables[place][vert]
    
    solver.Add(exp <= 1)

# Use exactly one vertex at each place in the path (terminal vertex included)
for vars in variables:
    solver.Add(sum(vars) == 1)

# Allow only usage of existing edges

for place in range(M - 1):
    for vert1 in range(N+1):
        for vert2 in range(N+1):

            if vert1 == N:
                if vert2 != N:
                    solver.Add(variables[place][vert1] + variables[place + 1][vert2] <= 1)
            else:
                if vert2 != N and vert1 != vert2:
                    if not adjacency_matrix[vert1][vert2]:
                        solver.Add(variables[place][vert1] + variables[place + 1][vert2] <= 1)


In [38]:
print(f"Solving with {solver.SolverVersion()}")

start = time.time()
status = solver.Solve()
end = time.time()
print(f"Solver took : {end - start} seconds")

if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    print("Objective value =", solver.Objective().Value())
    for vars in variables:
        for var in vars:
            print(f"{var.name} = {var.solution_value()}")
else:
    print("The problem does not have an optimal solution.")

Solving with CP-SAT solver v9.9.3963
Solver took : 3.6856436729431152 seconds
Solution:
Objective value = 100.0
<bound method Variable.name of X[0, 0]> = 0.0
<bound method Variable.name of X[0, 1]> = 0.0
<bound method Variable.name of X[0, 2]> = 0.0
<bound method Variable.name of X[0, 3]> = 0.0
<bound method Variable.name of X[0, 4]> = 0.0
<bound method Variable.name of X[0, 5]> = 0.0
<bound method Variable.name of X[0, 6]> = 0.0
<bound method Variable.name of X[0, 7]> = 0.0
<bound method Variable.name of X[0, 8]> = 0.0
<bound method Variable.name of X[0, 9]> = 0.0
<bound method Variable.name of X[0, 10]> = 0.0
<bound method Variable.name of X[0, 11]> = 0.0
<bound method Variable.name of X[0, 12]> = 0.0
<bound method Variable.name of X[0, 13]> = 0.0
<bound method Variable.name of X[0, 14]> = 0.0
<bound method Variable.name of X[0, 15]> = 0.0
<bound method Variable.name of X[0, 16]> = 0.0
<bound method Variable.name of X[0, 17]> = 0.0
<bound method Variable.name of X[0, 18]> = 0.0
<boun